In [1]:
import pandas as pd
import numpy as np
import glob
import plotly.graph_objects as go

In [2]:
# functions written by Mitali and Girish in 2024
# adapted by Tanvi Deora, April 2025

def theta_phi(x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4):
    """
    Calculate the angles theta and phi between two vectors

    Parameters:
        x1,y1,z1 (float) : The coordinates of point 1.
        x2,y2,z2 (float) : The coordinates of point 2.
        x3,y3,z3 (float) : The coordinates of point 3.
        x4,y4,z4 (float) : The coordinates of point 4.

    Returns:
        theta (float) : The angle (in degrees) between point1, point 2 and point3 in the XY plane.
        phi (float) : The angle (in degrees) between point1, point 2 and point3 in the ZY plane.

    Dependencies:
        - numpy (np): Required for numerical computations and array operations.
    """

    # Convert the input coordinates to NumPy arrays for vector operations
    a = np.array([x1,y1,z1]) # head
    b = np.array([x2,y2,z2]) # abdomen
    c = np.array([x3,y3,z3]) # COM of leading (chasee) fly
    d = np.array([x4,y4,z4]) # COM of lagging (chaser) fly
    

    # Calculate the vectors
    ab = a - b # changed this from ba = a -b for consistency 
    cd = c - d 

    # Calculate the theta angle (angle in XY plane)
    theta = np.degrees(np.arctan2(np.cross(ab[:2], cd[:2]), np.dot(ab[:2], cd[:2])))

    # Calculate the phi angle (angle in ZY plane) 
    phi = np.degrees(np.arctan2(np.cross(ab[[0, 2]], cd[[0, 2]]), np.dot(ab[[0, 2]], cd[[0, 2]])))

    return theta, phi

In [3]:
# files = glob.glob('./Data/Outputs/Sampled_1k/COM/*_smoothened_*COM.csv')
files = glob.glob('../dataFolders/MuscaChasingMusca/Outputs\\test_spiral_COM.csv')

In [5]:
# info = pd.read_csv('./Data/Outputs/Raw_xyz_points/metadata.csv')
# info

In [6]:
for f in files:
    name = f.split('\\')[-1][:-4]
    df = pd.read_csv(f)
    
    framerate = 1000 # for sequences sampled at 1k
#     framerate = [row['framerate'] for ind, row in info.iterrows() if row['filename'][:-10] in f.split('\\')[-1]][0] 
    
    # get points to compute error angles
    
    # Method 2 - use tangent at a given point on the chaser trajectory to get heading vector, at the instants measured 
    # on the video
    
    # point connecting the the chaser and chasee
    chasee_COM = df.loc[1:, ['chasee_com_x', 'chasee_com_y', 'chasee_com_z']]
    chaser_COM = df.loc[1:, ['chaser_com_x', 'chaser_com_y', 'chaser_com_z']]
    
    # heading vector for chaser
    # compute tangent at each point on the chaser trajectory
    
    # Store unit tangent vectors
    unit_tangents = []
    
    # X-Y plane
    x_vals = np.diff(df.loc[:, 'chaser_com_x'])
    y_vals = np.diff(df.loc[:, 'chaser_com_y']) 
    
    # Approximate slope using central differences
    for i in range(len(x_vals)):
        x0, y0 = x_vals[i], y_vals[i]

        # Compute slope using finite difference
        if 0 < i < len(x_vals) - 1:
            dx = x_vals[i+1] - x_vals[i-1]
            dy = y_vals[i+1] - y_vals[i-1]
        elif i == 0:
            dx = x_vals[i+1] - x_vals[i]
            dy = y_vals[i+1] - y_vals[i]
        else:  # i == last index
            dx = x_vals[i] - x_vals[i-1]
            dy = y_vals[i] - y_vals[i-1]
        
        # Normalize to unit vector
        norm = np.sqrt(dx**2 + dy**2)
        if norm == 0:
            unit_dx, unit_dy = 0.0, 0.0  # Avoid division by zero
        else:
            unit_dx = dx / norm
            unit_dy = dy / norm

        unit_tangents.append({'x0': x0, 'y0': y0, 'dx': unit_dx, 'dy': unit_dy})
     
    
#     #Method 3 - use the last two points and assume a straight vector joining those points
#     chaser_com_current = df.loc[:-1, ['chaser_com_x', 'chaser_com_y', 'chaser_com_z']]
#     chaser_com_previous = df.loc[1:,['chaser_com_X', 'chaser_com_y', 'chaser_com_z']]
    
    for all_vectors in [chasee_COM, chaser_COM, chaser_com_current, chaser_com_previous]:
        print(all_vectors.isnull().values.any())
    
        
    chase_theta = []
    chase_phi = []
    for ind, _ in chasee_COM.iterrows():
        pt3 = chasee_COM.loc[ind, :]
        pt4 = chaser_COM.loc[ind, :]
        
        pt1 = chaser_head.loc[ind, :]
        pt2 = chaser_abdo.loc[ind, :]
        
        tt, ph = theta_phi(
            pt1['chaser_head_X'], pt1['chaser_head_Y'], pt1['chaser_head_Z'],
            pt2['chaser_abdomen_X'], pt2['chaser_abdomen_Y'], pt2['chaser_abdomen_Z'],
            pt3['chasee_com_x'], pt3['chasee_com_y'], pt3['chasee_com_z'],
            pt4['chaser_com_x'], pt4['chaser_com_y'], pt4['chaser_com_z']
        )
        
           
        chase_theta.append(tt)
        chase_phi.append(ph)
        
       
    
    #Calculating angular velocity of Chasee and chaser
    angvel_chase_theta = np.diff(chase_theta)*framerate
    angvel_chase_phi = np.diff(chase_phi)*framerate
    angvel_chase_theta = np.append(angvel_chase_theta, np.repeat(np.nan, 1))
    angvel_chase_phi = np.append(angvel_chase_phi, np.repeat(np.nan, 1))
    
    #Calculating angular accerelation of Chasee and chaser
    angacc_chase_theta = np.diff(angvel_chase_theta)
    angacc_chase_phi = np. diff(angvel_chase_phi)
    angacc_chase_theta = np.append(angacc_chase_theta, np.repeat(np.nan, 1))
    angacc_chase_phi = np.append(angacc_chase_phi, np.repeat(np.nan, 1))
  
    
    chase_df = pd.DataFrame({'chase_theta': chase_theta,
                            'chase_phi': chase_phi,
                            'chase_theta_speed': angvel_chase_theta,
                             'chase_phi_speed': angvel_chase_phi,
                             'chase_theta_acc': angacc_chase_theta,
                             'chase_phi_acc': angacc_chase_phi,
                            })
#     chase_df.to_csv('./Data/Outputs/Sampled_1k/Chase/' + name + '_ChaseErrorAnglesSpeedAcc.csv')
    chase_df.to_csv('../dataFolders/MuscaChasingMusca/Outputs/' + name + '_ChaseErrorAnglesSpeedAcc.csv')

KeyError: "None of [Index(['chaser_head_X', 'chaser_head_Y', 'chaser_head_Z'], dtype='object')] are in the [columns]"